<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/GPT2_Chatbot_Pruning_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jul 10 06:05:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tqdm
!pip install kobert-transformers==0.4.1
!pip install kogpt2-transformers==0.3.0
!pip install transformers==3.0.2
!pip install torch
!pip install tokenizers==0.8.1rc1
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.7 MB/s 
     |████████████████████████████████| 101 kB 9.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
     |████████████████████████████████| 596 kB 89.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 6.9 MB/s 
     |████████████████████████████████| 3.0 MB 69.6 MB/s 
     |████████████████████████████████| 880 kB 76.7 MB/s 
     |████████████████████████████████| 1.2 MB 76.7 

In [ ]:
# 구글 드라이브 연결
import os
import numpy as np
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/gpt"

Mounted at /content/gdrive/


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from kogpt2_transformers import get_kogpt2_tokenizer, get_kogpt2_model
import torch.nn.utils.prune as prune

In [ ]:
class DialogKoGPT2(nn.Module):
  def __init__(self):
    super(DialogKoGPT2, self).__init__()
    self.kogpt2 = get_kogpt2_model()

  def generate(self,
               input_ids,
               do_sample=True,
               max_length= 60,
               top_p=0.92,
               top_k=50,
               temperature= 0.6,
               no_repeat_ngram_size = None,
               num_return_sequences=2,
               early_stopping=False,
               ):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_p = top_p,
               top_k=top_k,
               temperature=temperature,
               no_repeat_ngram_size= no_repeat_ngram_size,
               num_return_sequences=num_return_sequences,
               early_stopping = early_stopping,
              )

  def forward(self, input, labels = None):
    if labels is not None:
      outputs = self.kogpt2(input, labels=labels)
    else:
      outputs = self.kogpt2(input)

    return outputs

In [ ]:
# Global Pruning 적용
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

model = DialogKoGPT2()
model.to(device)

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (model.kogpt2.transformer.h[i].attn.c_attn, 'weight'),
        (model.kogpt2.transformer.h[i].attn.c_proj, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

In [ ]:
# Global Pruning을 통해 전체 Sparsity를 20%로 만들 수 있음을 알 수 있음.

for i in range(12):
    print(
        "Sparsity in Layer {}-th c_attn weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(model.kogpt2.transformer.h[i].attn.c_attn.weight == 0))
            / float(model.kogpt2.transformer.h[i].attn.c_attn.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th c_proj weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(model.kogpt2.transformer.h[i].attn.c_proj.weight == 0))
            / float(model.kogpt2.transformer.h[i].attn.c_proj.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(model.kogpt2.transformer.h[i].attn.c_attn.weight == 0)
    numerator += torch.sum(model.kogpt2.transformer.h[i].attn.c_proj.weight == 0)

    denominator += model.kogpt2.transformer.h[i].attn.c_attn.weight.nelement()
    denominator += model.kogpt2.transformer.h[i].attn.c_proj.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th c_attn weight: 28.47%
Sparsity in Layer 1-th c_proj weight: 65.74%

Sparsity in Layer 2-th c_attn weight: 20.61%
Sparsity in Layer 2-th c_proj weight: 24.94%

Sparsity in Layer 3-th c_attn weight: 18.73%
Sparsity in Layer 3-th c_proj weight: 27.17%

Sparsity in Layer 4-th c_attn weight: 17.68%
Sparsity in Layer 4-th c_proj weight: 23.86%

Sparsity in Layer 5-th c_attn weight: 17.33%
Sparsity in Layer 5-th c_proj weight: 21.64%

Sparsity in Layer 6-th c_attn weight: 17.04%
Sparsity in Layer 6-th c_proj weight: 19.34%

Sparsity in Layer 7-th c_attn weight: 17.24%
Sparsity in Layer 7-th c_proj weight: 18.92%

Sparsity in Layer 8-th c_attn weight: 17.60%
Sparsity in Layer 8-th c_proj weight: 19.29%

Sparsity in Layer 9-th c_attn weight: 16.98%
Sparsity in Layer 9-th c_proj weight: 19.14%

Sparsity in Layer 10-th c_attn weight: 17.09%
Sparsity in Layer 10-th c_proj weight: 17.16%

Sparsity in Layer 11-th c_attn weight: 17.65%
Sparsity in Layer 11-th c_proj weight: 15.

In [ ]:
root_path=path
data_path = f"{root_path}/data/wellness_dialog_for_autoregressive.txt"
checkpoint_path =f"{root_path}/checkpoint"
# save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive-20210922-add-chatbotdata.pth"
save_ckpt_path = f"{checkpoint_path}/kogpt2-20220710-add-chatbotdata.pth"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

tokenizer = get_kogpt2_tokenizer()

count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

In [ ]:
while 1:
#for i in range(5):
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  tokenized_indexs = tokenizer.encode(sent)

  input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0).to(device)
  # set top_k to 50
  sample_output = model.generate(
      input_ids=input_ids
      ).to(device)


  print("Answer: " + tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True))
  print(100 * '-')
  # print(sample_output.shape)

# for s in kss.split_sentences(sent):
#     print(s)

Question: 요즘 기분이 우울한 느낌이에요.


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 우울함은 저절로 없어지기도 하지만 그렇지 않을 때도 있어요. 그럴 때는 전문가에게 도움을 요청하는 것도 좋은 방법이에요. 도움을 받는 것도 좋지만 이렇게 우울한 기분이 오래 지속되면 정말 힘들죠. 우울한 기분이 오래 지속되면 정말 힘들죠
----------------------------------------------------------------------------------------------------
Question: 요새 힘들어요


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 힘들면 울어도 돼요. 티슈를 준비해드릴까요?용기내서 이야기해보세요. 듣고 있으니 조금만 더 기다려주세요. 용기를 내보세요. 말씀해주세요. 더 들려주세요. 더 들려
----------------------------------------------------------------------------------------------------
Question: 너가 좋아하는건 뭐니?


...

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 지금도 늦지 않았어요.조금 더 마음을 달리 가져보세요.지금도 늦지 않았어요.지금도 늦지 않았어요.마음이 조금 차분해 질 거예요.지금도 늦지 않아요.
----------------------------------------------------------------------------------------------------
Question: 위로해줘


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 제가 위로해 드릴게요. 그럼 안녕히 주무세요. 정말 보고 싶으셨나봐요. 저한테 계속 말씀해주세요. 그럼 수줍게 웃어주세요. 그러면 더 많이 힘들 거예요. 그러면
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored